In [1]:
import cv2
import numpy as np
from deepface import DeepFace
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import time

# Create our custom emotion detection model
def create_custom_model(input_shape=(48, 48, 1)):
    model = Sequential()
    
    # First Convolutional Block
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Second Convolutional Block
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # 10 emotions: 7 basic + 3 custom
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
    
    return model

# Initialize the face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Extended emotion labels including our custom states
emotion_labels = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral',
    7: 'Excited',
    8: 'Confident',
    9: 'Nervous',
    10: 'Confused',
    11: 'Blanked-out'
}

class EmotionAnalyzer:
    def __init__(self):
        self.prev_emotions = []
        self.emotion_history = []
        self.face_time = 0
        self.no_face_time = 0
        self.last_major_movement = time.time()
        self.prev_face_position = None
        self.movement_threshold = 30  # pixels
        self.blink_count = 0
        self.last_blink_time = time.time()
        self.eye_aspect_ratio_threshold = 0.3
        
    def detect_custom_states(self, face_img, emotion_probs, face_position):
        # Get the base emotion and probabilities
        emotions = {}
        for emotion, score in emotion_probs.items():
            emotions[emotion] = score
            
        # Sort emotions by probability
        sorted_emotions = sorted(emotions.items(), key=lambda x: x[1], reverse=True)
        base_emotion = sorted_emotions[0][0]
        
        current_time = time.time()
        
        # Add the current base emotion to our history
        self.emotion_history.append(base_emotion)
        if len(self.emotion_history) > 30:  # Keep last 30 frames (about 1 second)
            self.emotion_history.pop(0)
        
        # Calculate movement
        movement = 0
        if self.prev_face_position is not None:
            movement = np.sqrt((face_position[0] - self.prev_face_position[0])**2 + 
                              (face_position[1] - self.prev_face_position[1])**2)
            
            if movement > self.movement_threshold:
                self.last_major_movement = current_time
        
        self.prev_face_position = face_position
        
        # Time since last major movement
        time_since_movement = current_time - self.last_major_movement
        
        # Analyze emotion patterns
        emotion_counts = {}
        for emotion in self.emotion_history:
            if emotion in emotion_counts:
                emotion_counts[emotion] += 1
            else:
                emotion_counts[emotion] = 1
        
        dominant_emotion = max(emotion_counts, key=emotion_counts.get) if emotion_counts else None
        emotion_stability = emotion_counts.get(dominant_emotion, 0) / len(self.emotion_history) if self.emotion_history else 0
        
        # Detect custom states based on heuristics
        custom_emotion = base_emotion
        custom_prob = emotions[base_emotion]
        
        if base_emotion == 'happy' and movement > self.movement_threshold * 1.5:
            custom_emotion = 'Excited'
            custom_prob = emotions[base_emotion] * 0.9  # Slightly reduce confidence for derived state
        
        elif emotion_stability > 0.7 and base_emotion in ['neutral', 'happy'] and time_since_movement > 2:
            custom_emotion = 'Confident'
            custom_prob = emotions[base_emotion] * 0.85
        
        elif base_emotion in ['fear', 'surprise'] and movement > self.movement_threshold:
            custom_emotion = 'Nervous'
            custom_prob = emotions[base_emotion] * 0.9
        
        elif len(set(self.emotion_history[-10:])) >= 4:  # Rapidly changing emotions
            custom_emotion = 'Confused'
            custom_prob = 0.7  # Assign a reasonable confidence
        
        elif time_since_movement > 5 and base_emotion == 'neutral':
            custom_emotion = 'Blanked-out'
            custom_prob = 0.75
        
        # Create a new dictionary with all emotions including our custom ones
        result = dict(emotions)
        
        # Only add the custom emotion if it's different from the base emotion
        if custom_emotion != base_emotion:
            result[custom_emotion] = custom_prob
            
        # Sort all emotions by probability
        sorted_result = sorted(result.items(), key=lambda x: x[1], reverse=True)
        
        return sorted_result

def main():
    cap = cv2.VideoCapture(0)
    analyzer = EmotionAnalyzer()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Mirror the image horizontally
        frame = cv2.flip(frame, 1)
            
        # Convert to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            # Extract face ROI
            face_roi = frame[y:y+h, x:x+w]
            
            try:
                # Use DeepFace for base emotion detection
                analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                
                # Get the emotion probabilities
                if isinstance(analysis, list):
                    emotion_probs = analysis[0]['emotion']
                else:
                    emotion_probs = analysis['emotion']
                
                # Detect custom emotional states
                face_position = (x + w/2, y + h/2)  # Center of the face
                custom_emotions = analyzer.detect_custom_states(face_roi, emotion_probs, face_position)
                
                # Draw rectangle around face
                color = (0, 255, 0)  # Green
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                
                # Display top emotions
                if len(custom_emotions) >= 2:
                    top_emotion = custom_emotions[0]
                    second_emotion = custom_emotions[1]
                    
                    # Only show both if second emotion has probability > 37%
                    if second_emotion[1] > 37:
                        emotion_text = f"{top_emotion[0]}: {top_emotion[1]:.1f}%, {second_emotion[0]}: {second_emotion[1]:.1f}%"
                    else:
                        emotion_text = f"{top_emotion[0]}: {top_emotion[1]:.1f}%"
                else:
                    emotion_text = f"{custom_emotions[0][0]}: {custom_emotions[0][1]:.1f}%"
                
                cv2.putText(frame, emotion_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
            except Exception as e:
                print(f"Error in emotion detection: {e}")
                # Just draw the face rectangle if emotion detection fails
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        # Display the resulting frame
        cv2.imshow('Advanced Emotion Detection', frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
